<a href="https://colab.research.google.com/github/Kotwic4/fake-news-detector/blob/colab/colab/flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install vaderSentiment==3.2.1 requests numpy matplotlib pandas flair

In [0]:
!nvidia-smi

Wed Dec  4 11:37:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!wget - O politifact_fake.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
!wget - O politifact_real.csv https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_real.csv

--2019-12-04 11:37:47--  http://-/
Resolving - (-)... failed: Name or service not known.
wget: unable to resolve host address ‘-’
--2019-12-04 11:37:47--  http://o/
Resolving o (o)... failed: Name or service not known.
wget: unable to resolve host address ‘o’
--2019-12-04 11:37:47--  http://politifact_fake.csv/
Resolving politifact_fake.csv (politifact_fake.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘politifact_fake.csv’
--2019-12-04 11:37:47--  https://raw.githubusercontent.com/KaiDMML/FakeNewsNet/master/dataset/politifact_fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3286418 (3.1M) [text/plain]
Saving to: ‘politifact_fake.csv.1’

politifact_fake.csv 100%[===================>]   3.13M  --.-KB/s    in 0.05s   

2019

In [0]:
!ls -l

total 22604
-rw-r--r-- 1 root root 3286418 Dec  4 11:35 politifact_fake.csv
-rw-r--r-- 1 root root 3286418 Dec  4 11:37 politifact_fake.csv.1
-rw-r--r-- 1 root root 8278658 Dec  4 11:35 politifact_real.csv
-rw-r--r-- 1 root root 8278658 Dec  4 11:37 politifact_real.csv.1
drwxr-xr-x 1 root root    4096 Nov 21 16:30 sample_data


In [0]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from flair.datasets import CSVClassificationCorpus
import os
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

In [0]:
DATASET_NAME = 'politifact'
DATASET_PATH = './{}'.format(DATASET_NAME)
REAL_DATA_PATH = '{}_real.csv'.format(DATASET_PATH)
FAKE_DATA_PATH = '{}_fake.csv'.format(DATASET_PATH)

In [0]:
fake_arts = pd.read_csv(FAKE_DATA_PATH, na_values=['nan'], keep_default_na=False)
real_arts = pd.read_csv(REAL_DATA_PATH, na_values=['nan'], keep_default_na=False)

In [0]:
def parse_art_data_frame(df):
    return [{'id': id, 'url': url, 'title': title} for id, url, title, tweets in df.values]
    
fake_arts_with_content = parse_art_data_frame(fake_arts)
real_arts_with_content = parse_art_data_frame(real_arts)

In [0]:
fake_data = [(art, 'fake') for art in fake_arts_with_content]
real_data = [(art, 'real') for art in real_arts_with_content]

In [0]:
all_data = fake_data + real_data
X = [x for x,y in all_data]
y = [y for x,y in all_data]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [0]:
print(len(train_data))
print(len(test_data))
print(len(train_data) + len(test_data))
print(len(fake_data) + len(real_data))

844
212
1056
1056


In [0]:
print(len([y for y in y_train if y == 'fake']))
print(len([y for y in y_test if y == 'fake']))

print(len([y for y in y_train if y == 'real']))
print(len([y for y in y_test if y == 'real']))

334
98
510
114


In [0]:
def clear_text(text):
    return ' '.join(text.split())

class Classifier():
    def __init__(self, classifier):
        self.classifier = classifier
        
    def predict(self, text):
        text = clear_text(text)
        sentence = Sentence(text)
        self.classifier.predict(sentence)
        return sentence.labels[0]

def transform_data(data):
    return [{'label': label, 'text': clear_text(x)} for x, label in data]

def save_data(data, data_folder = '.'):
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    data = transform_data(data)
    frame_data = pd.DataFrame(data)
    train_path = '{}/train.csv'.format(data_folder)
    test_path = '{}/test.csv'.format(data_folder)
    dev_path = '{}/dev.csv'.format(data_folder)
    frame_data.iloc[0:int(len(data)*0.8)].to_csv(train_path, sep='\t', index = False, header = False)
    frame_data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(test_path, sep='\t', index = False, header = False)
    frame_data.iloc[int(len(data)*0.9):].to_csv(dev_path, sep='\t', index = False, header = False)

def load_corpus(data_folder = '.'):
    column_name_map = {1: "text", 0: "label"}
    return CSVClassificationCorpus(data_folder,
                                     column_name_map,
                                     delimiter='\t',
                                  test_file='test.csv',
                                  dev_file='dev.csv',
                                  train_file='train.csv')
    
def train_classifier(corpus, model_folder = '.', max_epochs = 1):
    label_dict = corpus.make_label_dictionary()

    word_embeddings = [
        WordEmbeddings('glove'),
        FlairEmbeddings('news-forward-fast'),
        FlairEmbeddings('news-backward-fast')
    ]

    document_embeddings = DocumentRNNEmbeddings(word_embeddings,
                                                hidden_size=512,
                                                reproject_words=True,
                                                reproject_words_dimension=256)

    classifier = TextClassifier(document_embeddings,
                                label_dictionary=label_dict)

    trainer = ModelTrainer(classifier, corpus)

    trainer.train(model_folder, max_epochs=max_epochs)
    
    return TextClassifier.load('{}/best-model.pt'.format(model_folder))
    
def train_model(train_data,
               data_folder = '.',
               model_folder = '.',
               max_epochs=1
               ):
    save_data(train_data, data_folder)
    corpus = load_corpus(data_folder)
    classifier = train_classifier(corpus, model_folder, max_epochs)
    return Classifier(classifier)

def calculate_metrics(y_true, y_pred, pos_label = 'fake'):
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label)
    recall = recall_score(y_true, y_pred, pos_label=pos_label)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label)
    return acc, precision, recall, f1

def validate_model(test_data, classifier):
    y_true = [label for x, label in test_data]
    y_pred = [classifier.predict(x).value for x, label in test_data]
    acc, precision, recall, f1 = calculate_metrics(y_true, y_pred)
    print("acc: ", acc)
    print("precision: ", precision)
    print("recall: ", recall)
    print("f1: ", f1)
    return acc, precision, recall, f1

def make_test(train_data, test_data, data_folder, model_folder, max_epochs):
    classifier = train_model(train_data, data_folder, model_folder, max_epochs)
    validate_model(test_data, classifier)

In [0]:
train_content = [(x, label) for x, label in train_data] 
test_content = [(x, label) for x, label in test_data]

In [0]:
train_title = [(x['title'], label) for x, label in train_content]
test_title = [(x['title'], label) for x, label in test_content]
print(len([x for x, label in train_title if x == '']))
print(len([x for x, label in test_title if x == '']))
train_title = [(x, label) for x, label in train_title if x != ''] 
test_title = [(x, label) for x, label in test_title if x != '']
print(len(train_title))
print(len(test_title))

0
0
844
212


In [0]:
path = './test_csv/title1'
make_test(train_title, test_title, path, path, 1)

2019-12-04 11:38:00,013 Reading data from test_csv/title1
2019-12-04 11:38:00,015 Train: test_csv/title1/train.csv
2019-12-04 11:38:00,017 Dev: test_csv/title1/dev.csv
2019-12-04 11:38:00,019 Test: test_csv/title1/test.csv
2019-12-04 11:38:00,027 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:00<00:00, 1640.17it/s]

2019-12-04 11:38:00,558 [b'real', b'fake']


2019-12-04 11:38:00,653 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpgpbpsbj0


100%|██████████| 160000128/160000128 [00:03<00:00, 49344520.30B/s]

...

100%|██████████| 19689779/19689779 [00:00<00:00, 45338737.73B/s]

2019-12-04 11:38:14,271 copying /tmp/tmpxf3r4qy_ to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2019-12-04 11:38:14,299 removing temp file /tmp/tmpxf3r4qy_


2019-12-04 11:38:14,444 ----------------------------------------------------------------------------------------------------
2019-12-04 11:38:14,445 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:38:25,581 ----------------------------------------------------------------------------------------------------
2019-12-04 11:38:25,582 Testing using best model ...
2019-12-04 11:38:25,589 loading file test_csv/title1/best-model.pt
2019-12-04 11:38:27,432 0.5952	0.5952	0.5952
2019-12-04 11:38:27,434 
MICRO_AVG: acc 0.4237 - f1-score 0.5952
MACRO_AVG: acc 0.4223 - f1-score 0.5931500000000001
fake       tp: 28 - fp: 31 - fn: 3 - tn: 22 - precision: 0.4746 - recall: 0.9032 - accuracy: 0.4516 - f1-score: 0.6222
real       tp: 22 - fp: 3 - fn: 31 - tn: 28 - precision: 0.8800 - recall: 0.4151 - accuracy: 0.3929 - f1-score: 0.5641
2019-12-04 11:38:27,441 ----------------------------------------------------------------------------------------------------
2019-12-04 11:38:27,445 loading file ./test_csv/title1/best-model.pt
acc:  0.660377358490566
precision:  0.5878378378378378
recall:  0.8877551020408163
f1:  0.7073170731707316


In [0]:
path = './test_csv/title10'
make_test(train_title, test_title, path, path, 10)

2019-12-04 11:38:30,946 Reading data from test_csv/title10
2019-12-04 11:38:30,947 Train: test_csv/title10/train.csv
2019-12-04 11:38:30,948 Dev: test_csv/title10/dev.csv
2019-12-04 11:38:30,952 Test: test_csv/title10/test.csv
2019-12-04 11:38:30,960 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:00<00:00, 1150.22it/s]

2019-12-04 11:38:31,883 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:38:33,250 ----------------------------------------------------------------------------------------------------
2019-12-04 11:38:33,252 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:38:41,165 ----------------------------------------------------------------------------------------------------
2019-12-04 11:38:41,750 epoch 2 - iter 0/22 - loss 0.55306464 - samples/sec: 503.58
2019-12-04 11:38:42,012 epoch 2 - iter 2/22 - loss 0.46997536 - samples/sec: 277.88
2019-12-04 11:38:42,246 epoch 2 - iter 4/22 - loss 0.49628988 - samples/sec: 302.44
2019-12-04 11:38:42,487 epoch 2 - iter 6/22 - loss 0.48343866 - samples/sec: 296.25
2019-12-04 11:38:42,837 epoch 2 - iter 8/22 - loss 0.50570683 - samples/sec: 213.95
2019-12-04 11:38:43,093 epoch 2 - iter 10/22 - loss 0.48435369 - samples/sec: 284.20
2019-12-04 11:38:43,352 epoch 2 - iter 12/22 - loss 0.47704445 - samples/sec: 270.01
2019-12-04 11:38:43,597 epoch 2 - iter 14/22 - loss 0.48344585 - samples/sec: 305.14
2019-12-04 11:38:43,919 epoch 2 - iter 16/22 - loss 0.48895950 - samples/sec: 216.85
2019-12-04 11:38:44,210 epoch 2 - iter 18/22 - loss 0.48897575 - samples/sec: 241.49
2019-12-04 11:38:44,509 epoch 

In [0]:
train_url = [(x['url'], label) for x, label in train_content] 
test_url = [(x['url'], label) for x, label in test_content]
print(len([x for x, label in train_url if x == '']))
print(len([x for x, label in test_url if x == '']))
train_url = [(x, label) for x, label in train_url if x != ''] 
test_url = [(x, label) for x, label in test_url if x != '']
print(len(train_url))
print(len(test_url))

39
22
805
190


In [0]:
path = './test_csv/url1'
make_test(train_url, test_url, path, path, 1)

2019-12-04 11:39:36,964 Reading data from test_csv/url1
2019-12-04 11:39:36,965 Train: test_csv/url1/train.csv
2019-12-04 11:39:36,967 Dev: test_csv/url1/dev.csv
2019-12-04 11:39:36,969 Test: test_csv/url1/test.csv
2019-12-04 11:39:36,977 Computing label dictionary. Progress:


100%|██████████| 644/644 [00:00<00:00, 884.97it/s] 

2019-12-04 11:39:38,165 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:39:39,856 ----------------------------------------------------------------------------------------------------
2019-12-04 11:39:39,858 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:39:51,840 ----------------------------------------------------------------------------------------------------
2019-12-04 11:39:51,842 Testing using best model ...
2019-12-04 11:39:51,848 loading file test_csv/url1/best-model.pt
2019-12-04 11:39:53,575 0.4375	0.4375	0.4375
2019-12-04 11:39:53,576 
MICRO_AVG: acc 0.28 - f1-score 0.4375
MACRO_AVG: acc 0.2447 - f1-score 0.36515000000000003
fake       tp: 31 - fp: 45 - fn: 0 - tn: 4 - precision: 0.4079 - recall: 1.0000 - accuracy: 0.4079 - f1-score: 0.5794
real       tp: 4 - fp: 0 - fn: 45 - tn: 31 - precision: 1.0000 - recall: 0.0816 - accuracy: 0.0816 - f1-score: 0.1509
2019-12-04 11:39:53,578 ----------------------------------------------------------------------------------------------------
2019-12-04 11:39:53,580 loading file ./test_csv/url1/best-model.pt
acc:  0.7526315789473684
precision:  0.6935483870967742
recall:  0.9052631578947369
f1:  0.7853881278538815


In [0]:
path = './test_csv/url10'
make_test(train_url, test_url, path, path, 10)

2019-12-04 11:39:57,665 Reading data from test_csv/url10
2019-12-04 11:39:57,668 Train: test_csv/url10/train.csv
2019-12-04 11:39:57,670 Dev: test_csv/url10/dev.csv
2019-12-04 11:39:57,676 Test: test_csv/url10/test.csv
2019-12-04 11:39:57,683 Computing label dictionary. Progress:


100%|██████████| 644/644 [00:00<00:00, 695.01it/s]

2019-12-04 11:39:59,165 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:40:00,132 ----------------------------------------------------------------------------------------------------
2019-12-04 11:40:00,134 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:40:09,606 ----------------------------------------------------------------------------------------------------
2019-12-04 11:40:10,392 epoch 2 - iter 0/21 - loss 0.48979393 - samples/sec: 300.75
2019-12-04 11:40:10,641 epoch 2 - iter 2/21 - loss 0.58826465 - samples/sec: 292.59
2019-12-04 11:40:10,905 epoch 2 - iter 4/21 - loss 0.55359312 - samples/sec: 281.93
2019-12-04 11:40:11,226 epoch 2 - iter 6/21 - loss 0.51095067 - samples/sec: 217.13
2019-12-04 11:40:11,546 epoch 2 - iter 8/21 - loss 0.52291862 - samples/sec: 216.65
2019-12-04 11:40:11,863 epoch 2 - iter 10/21 - loss 0.52413556 - samples/sec: 241.07
2019-12-04 11:40:12,145 epoch 2 - iter 12/21 - loss 0.51043547 - samples/sec: 247.80
2019-12-04 11:40:12,434 epoch 2 - iter 14/21 - loss 0.52980688 - samples/sec: 241.51
2019-12-04 11:40:12,717 epoch 2 - iter 16/21 - loss 0.55037225 - samples/sec: 254.70
2019-12-04 11:40:13,014 epoch 2 - iter 18/21 - loss 0.54938717 - samples/sec: 243.51
2019-12-04 11:40:13,202 epoch 

In [0]:
train_mix = [(x['url'] + ', ' + x['title'], label) for x, label in train_content] 
test_mix = [(x['url'] + ', ' + x['title'], label) for x, label in test_content]
print(len([x for x, label in train_mix if x == '']))
print(len([x for x, label in test_mix if x == '']))
train_mix = [(x, label) for x, label in train_mix if x != ''] 
test_mix = [(x, label) for x, label in test_mix if x != '']
print(len(train_mix))
print(len(test_mix))

0
0
844
212


In [0]:
path = './test_csv/mix1'
make_test(train_mix, test_mix, path, path, 1)

2019-12-04 11:41:13,677 Reading data from test_csv/mix1
2019-12-04 11:41:13,680 Train: test_csv/mix1/train.csv
2019-12-04 11:41:13,682 Dev: test_csv/mix1/dev.csv
2019-12-04 11:41:13,686 Test: test_csv/mix1/test.csv
2019-12-04 11:41:13,693 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:00<00:00, 719.56it/s] 

2019-12-04 11:41:15,261 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:41:16,270 ----------------------------------------------------------------------------------------------------
2019-12-04 11:41:16,271 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:41:31,860 ----------------------------------------------------------------------------------------------------
2019-12-04 11:41:31,861 Testing using best model ...
2019-12-04 11:41:31,862 loading file test_csv/mix1/best-model.pt
2019-12-04 11:41:34,886 0.6905	0.6905	0.6905
2019-12-04 11:41:34,889 
MICRO_AVG: acc 0.5273 - f1-score 0.6905
MACRO_AVG: acc 0.4161 - f1-score 0.5404
fake       tp: 5 - fp: 0 - fn: 26 - tn: 53 - precision: 1.0000 - recall: 0.1613 - accuracy: 0.1613 - f1-score: 0.2778
real       tp: 53 - fp: 26 - fn: 0 - tn: 5 - precision: 0.6709 - recall: 1.0000 - accuracy: 0.6709 - f1-score: 0.8030
2019-12-04 11:41:34,890 ----------------------------------------------------------------------------------------------------
2019-12-04 11:41:34,891 loading file ./test_csv/mix1/best-model.pt
acc:  0.5801886792452831
precision:  1.0
recall:  0.09183673469387756
f1:  0.16822429906542058


In [0]:
path = './test_csv/mix10'
make_test(train_mix, test_mix, path, path, 10)

2019-12-04 11:41:39,780 Reading data from test_csv/mix10
2019-12-04 11:41:39,781 Train: test_csv/mix10/train.csv
2019-12-04 11:41:39,783 Dev: test_csv/mix10/dev.csv
2019-12-04 11:41:39,784 Test: test_csv/mix10/test.csv
2019-12-04 11:41:39,791 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:01<00:00, 666.69it/s] 

2019-12-04 11:41:41,451 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:41:43,321 ----------------------------------------------------------------------------------------------------
2019-12-04 11:41:43,324 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:41:55,649 ----------------------------------------------------------------------------------------------------
2019-12-04 11:41:56,532 epoch 2 - iter 0/22 - loss 0.70023197 - samples/sec: 266.99
2019-12-04 11:41:56,942 epoch 2 - iter 2/22 - loss 0.61663314 - samples/sec: 167.88
2019-12-04 11:41:57,443 epoch 2 - iter 4/22 - loss 0.57541447 - samples/sec: 142.08
2019-12-04 11:41:57,901 epoch 2 - iter 6/22 - loss 0.53374495 - samples/sec: 151.75
2019-12-04 11:41:58,357 epoch 2 - iter 8/22 - loss 0.53108068 - samples/sec: 147.78
2019-12-04 11:41:58,912 epoch 2 - iter 10/22 - loss 0.52675792 - samples/sec: 134.41
2019-12-04 11:41:59,401 epoch 2 - iter 12/22 - loss 0.53958977 - samples/sec: 137.51
2019-12-04 11:41:59,871 epoch 2 - iter 14/22 - loss 0.52282524 - samples/sec: 148.49
2019-12-04 11:42:00,298 epoch 2 - iter 16/22 - loss 0.52217306 - samples/sec: 159.26
2019-12-04 11:42:00,795 epoch 2 - iter 18/22 - loss 0.51712364 - samples/sec: 136.71
2019-12-04 11:42:01,272 epoch 

In [0]:
path = './test_csv/title50'
make_test(train_title, test_title, path, path, 50)

2019-12-04 11:43:34,623 Reading data from test_csv/title50
2019-12-04 11:43:34,625 Train: test_csv/title50/train.csv
2019-12-04 11:43:34,626 Dev: test_csv/title50/dev.csv
2019-12-04 11:43:34,627 Test: test_csv/title50/test.csv
2019-12-04 11:43:34,634 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:00<00:00, 808.07it/s] 

2019-12-04 11:43:36,154 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:43:37,223 ----------------------------------------------------------------------------------------------------
2019-12-04 11:43:37,225 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:43:46,574 ----------------------------------------------------------------------------------------------------
2019-12-04 11:43:47,341 epoch 2 - iter 0/22 - loss 0.42676669 - samples/sec: 286.58
2019-12-04 11:43:47,716 epoch 2 - iter 2/22 - loss 0.46213836 - samples/sec: 204.57
2019-12-04 11:43:47,967 epoch 2 - iter 4/22 - loss 0.49009652 - samples/sec: 288.77
2019-12-04 11:43:48,206 epoch 2 - iter 6/22 - loss 0.50464803 - samples/sec: 297.92
2019-12-04 11:43:48,450 epoch 2 - iter 8/22 - loss 0.50143387 - samples/sec: 311.71
2019-12-04 11:43:48,671 epoch 2 - iter 10/22 - loss 0.51677523 - samples/sec: 323.67
2019-12-04 11:43:48,892 epoch 2 - iter 12/22 - loss 0.51237541 - samples/sec: 321.45
2019-12-04 11:43:49,093 epoch 2 - iter 14/22 - loss 0.49757318 - samples/sec: 384.27
2019-12-04 11:43:49,298 epoch 2 - iter 16/22 - loss 0.48894718 - samples/sec: 352.00
2019-12-04 11:43:49,542 epoch 2 - iter 18/22 - loss 0.48483993 - samples/sec: 297.70
2019-12-04 11:43:49,822 epoch 

In [0]:
path = './test_csv/url50'
make_test(train_url, test_url, path, path, 50)

2019-12-04 11:48:30,920 Reading data from test_csv/url50
2019-12-04 11:48:30,921 Train: test_csv/url50/train.csv
2019-12-04 11:48:30,923 Dev: test_csv/url50/dev.csv
2019-12-04 11:48:30,924 Test: test_csv/url50/test.csv
2019-12-04 11:48:30,932 Computing label dictionary. Progress:


100%|██████████| 644/644 [00:00<00:00, 817.20it/s] 

2019-12-04 11:48:32,383 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:48:34,270 ----------------------------------------------------------------------------------------------------
2019-12-04 11:48:34,272 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:48:43,592 ----------------------------------------------------------------------------------------------------
2019-12-04 11:48:44,527 epoch 2 - iter 0/21 - loss 1.04485035 - samples/sec: 391.85
2019-12-04 11:48:44,839 epoch 2 - iter 2/21 - loss 0.63812202 - samples/sec: 224.57
2019-12-04 11:48:45,193 epoch 2 - iter 4/21 - loss 0.56177261 - samples/sec: 196.61
2019-12-04 11:48:45,546 epoch 2 - iter 6/21 - loss 0.52640999 - samples/sec: 202.57
2019-12-04 11:48:45,911 epoch 2 - iter 8/21 - loss 0.52671319 - samples/sec: 190.18
2019-12-04 11:48:46,224 epoch 2 - iter 10/21 - loss 0.54110088 - samples/sec: 230.73
2019-12-04 11:48:46,563 epoch 2 - iter 12/21 - loss 0.54232086 - samples/sec: 207.22
2019-12-04 11:48:46,872 epoch 2 - iter 14/21 - loss 0.54985793 - samples/sec: 241.01
2019-12-04 11:48:47,188 epoch 2 - iter 16/21 - loss 0.54271087 - samples/sec: 241.24
2019-12-04 11:48:47,492 epoch 2 - iter 18/21 - loss 0.54659438 - samples/sec: 231.60
2019-12-04 11:48:47,676 epoch 

In [0]:
path = './test_csv/mix50'
make_test(train_mix, test_mix, path, path, 50)

2019-12-04 11:54:37,896 Reading data from test_csv/mix50
2019-12-04 11:54:37,897 Train: test_csv/mix50/train.csv
2019-12-04 11:54:37,898 Dev: test_csv/mix50/dev.csv
2019-12-04 11:54:37,900 Test: test_csv/mix50/test.csv
2019-12-04 11:54:37,907 Computing label dictionary. Progress:


100%|██████████| 675/675 [00:00<00:00, 682.15it/s] 

2019-12-04 11:54:39,613 [b'real', b'fake']



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


2019-12-04 11:54:42,158 ----------------------------------------------------------------------------------------------------
2019-12-04 11:54:42,162 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=2148, out_features=256, bias=True)
    (rnn): GRU(256, 512, ba

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DocumentRNNEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type StackedEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type WordEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type FlairEmbeddings. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be

2019-12-04 11:54:55,845 ----------------------------------------------------------------------------------------------------
2019-12-04 11:54:56,921 epoch 2 - iter 0/22 - loss 0.57385504 - samples/sec: 196.72
2019-12-04 11:54:57,468 epoch 2 - iter 2/22 - loss 0.47300846 - samples/sec: 128.89
2019-12-04 11:54:57,975 epoch 2 - iter 4/22 - loss 0.46829290 - samples/sec: 140.12
2019-12-04 11:54:58,440 epoch 2 - iter 6/22 - loss 0.49711214 - samples/sec: 145.85
2019-12-04 11:54:58,945 epoch 2 - iter 8/22 - loss 0.57512766 - samples/sec: 137.65
2019-12-04 11:54:59,513 epoch 2 - iter 10/22 - loss 0.59654569 - samples/sec: 120.90
2019-12-04 11:54:59,977 epoch 2 - iter 12/22 - loss 0.57034836 - samples/sec: 151.90
2019-12-04 11:55:00,374 epoch 2 - iter 14/22 - loss 0.55232813 - samples/sec: 170.88
2019-12-04 11:55:00,888 epoch 2 - iter 16/22 - loss 0.55167042 - samples/sec: 133.58
2019-12-04 11:55:01,350 epoch 2 - iter 18/22 - loss 0.53505326 - samples/sec: 148.73
2019-12-04 11:55:01,797 epoch 